<a href="https://colab.research.google.com/github/sajajjain/machine_learning_examples/blob/master/faster_rcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch as tc
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [2]:
image = tc.zeros((1,3,800,800)).float()
bbox = tc.FloatTensor([[20,30,400,500], [300,400,500, 600]])
# y1, x1, y2, x2
labels = tc.LongTensor([6,8])
sub_sample = 16

In [3]:
dummy_img = tc.zeros((1,3,800,800)).float()
print(dummy_img)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],

         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]])


In [6]:
model = torchvision.models.vgg16(pretrained = True)
fe = list(model.features)
print(fe)
len(fe)

[Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False), Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)), ReLU(inplace=True), Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1

31

In [5]:
req_feature = []
k = dummy_img.clone()
for i in fe:
  k = i(k)
  if k.size()[2]<800/16:
    break
  req_feature.append(i)
  out_channel = k.size()[1]

print(out_channel)
print(len(req_feature))

512
30


In [ ]:
faster_rcnn_fe_extractor = nn.Sequential(*req_feature)
faster_rcnn_fe_extractor

In [19]:
out_map = faster_rcnn_fe_extractor(image)


In [20]:
print(out_map.size())

torch.Size([1, 512, 50, 50])


In [ ]:
plt.imshow(out_map[0,10].detach().numpy())

In [24]:
ratios = [0.5, 1, 2]
anchor_scales = [8, 16, 32]
anchor_base = np.zeros((len(ratios)* len(anchor_scales), 4), dtype=np.float32)
print(anchor_base)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [28]:
ctr_y = sub_sample / 2
ctr_x = sub_sample / 2
print(ctr_y, ctr_x)
for i in range(len(ratios)):
  for j in range(len(anchor_scales)):
    h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i]) 
    w = sub_sample * anchor_scales[j] * np.sqrt(1./ratios[i])

    index = i * len(anchor_scales) + j

    anchor_base[index, 0] = ctr_y - h /2.
    anchor_base[index, 1] = ctr_x - w /2.
    anchor_base[index, 2] = ctr_y + h /2.
    anchor_base[index, 3] = ctr_x + w /2.
anchor_base

8.0 8.0


array([[ -37.254833,  -82.50967 ,   53.254833,   98.50967 ],
       [ -82.50967 , -173.01933 ,   98.50967 ,  189.01933 ],
       [-173.01933 , -354.03867 ,  189.01933 ,  370.03867 ],
       [ -56.      ,  -56.      ,   72.      ,   72.      ],
       [-120.      , -120.      ,  136.      ,  136.      ],
       [-248.      , -248.      ,  264.      ,  264.      ],
       [ -82.50967 ,  -37.254833,   98.50967 ,   53.254833],
       [-173.01933 ,  -82.50967 ,  189.01933 ,   98.50967 ],
       [-354.03867 , -173.01933 ,  370.03867 ,  189.01933 ]],
      dtype=float32)

In [44]:
fe_size = 800//16
ctr_x = np.arange(16, (fe_size + 1) * 16, 16)
ctr_y = np.arange(16, (fe_size + 1) * 16, 16)
ctr_x

array([ 16,  32,  48,  64,  80,  96, 112, 128, 144, 160, 176, 192, 208,
       224, 240, 256, 272, 288, 304, 320, 336, 352, 368, 384, 400, 416,
       432, 448, 464, 480, 496, 512, 528, 544, 560, 576, 592, 608, 624,
       640, 656, 672, 688, 704, 720, 736, 752, 768, 784, 800])

In [47]:
ctr = np.zeros((50*50, 2))
index = 0
for x in range(len(ctr_x)):
  for y in range(len(ctr_y)):
    ctr[index, 1] = ctr_x[x] - 8
    ctr[index, 0] = ctr_y[y] - 8
    index += 1
ctr

array([[  8.,   8.],
       [ 24.,   8.],
       [ 40.,   8.],
       ...,
       [760., 792.],
       [776., 792.],
       [792., 792.]])

In [48]:
anchors = np.zeros(((fe_size* fe_size *9), 4))
index = 0
for c in ctr:
  ctr_y, ctr_x = c
  for i in range(len(ratios)):
    for j in range(len(anchor_scales)):
      h = sub_sample * anchor_scales[j] * np.sqrt(ratios[i])
      w = sub_sample * anchor_scales[j] * np.sqrt(1./ ratios[i])

      anchors[index, 0] = ctr_y - h/2.
      anchors[index, 1] = ctr_x - w/2.
      anchors[index, 2] = ctr_y + h/2.
      anchors[index, 3] = ctr_x + w/2.


In [ ]:
! git clone 